# **Homework**: Data talks club data engineering zoomcamp Data loading workshop
## By Edgar Cusi

> Bloque con sangría



Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [10]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
sum_outputs = 0
i=1
for sqrt_value in generator:
    sum_outputs = sum_outputs + sqrt_value
    print(i,'total :', sqrt_value)
    i=i+1
print('total :', sum_outputs)

1 total : 1.0
2 total : 1.4142135623730951
3 total : 1.7320508075688772
4 total : 2.0
5 total : 2.23606797749979
6 total : 2.449489742783178
7 total : 2.6457513110645907
8 total : 2.8284271247461903
9 total : 3.0
10 total : 3.1622776601683795
11 total : 3.3166247903554
12 total : 3.4641016151377544
13 total : 3.605551275463989
total : 32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [39]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


def calcular_suma_total_edades(personas_1, personas_2):
  suma_total_edades = 0
  for personas in (personas_1, personas_2):
    for persona in personas:
      suma_total_edades += persona["Age"]
  return suma_total_edades


suma_total_edades = calcular_suma_total_edades(people_1(), people_2())

print(f"Suma total de edades: {suma_total_edades}")




{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Suma total de edades: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [40]:


def fuse_by_id(gen1, gen2):
  merged = {}
  for gen in (gen1, gen2):
    for person in gen:
      id_ = person["ID"]
      merged.setdefault(id_, person).update(person)
  return merged.values()

def sum_ages(people):
  return sum(person["Age"] for person in people)

fused_people = fuse_by_id(people_1(), people_2())

#suma de edades
suma_edades = sum_ages(fused_people)

print(f"Suma de edades: {suma_edades}")

Suma de edades: 266


# Solution: First make sure that the following modules are installed:

In [13]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [18]:
import requests

BASE_API_URL = "https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api"

# I call this a paginated getter
# as it's a function that gets data
# and also paginates until there is no more data
# by yielding pages, we "microbatch", which speeds up downstream processing

def paginated_getter():
    page_number = 1

    while True:
        # Set the query parameters
        params = {'page': page_number}

        # Make the GET request to the API
        response = requests.get(BASE_API_URL, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        page_json = response.json()
        print(f'got page number {page_number} with {len(page_json)} records')

        # if the page has no records, stop iterating
        if page_json:
            yield page_json
            page_number += 1
        else:
            # No more data, break the loop
            break

if __name__ == '__main__':
    # Use the generator to iterate over pages
    for page_data in paginated_getter():
        # Process each page as needed
        print(page_data)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
import requests
import json


url = "https://storage.googleapis.com/dtc_zoomcamp_api/yellow_tripdata_2009-06.jsonl"

def stream_download_jsonl(url):
    response = requests.get(url,stream=True)
    response.raise_for_status()  # Raise an HTTPError for bad responses
    for line in response.iter_lines():
        if line:
            yield json.loads(line)

import time
start = time.time()

row_counter = 0
for row in stream_download_jsonl(url):
    print(row)
    row_counter +=1
    if row_counter >= 5:
        break;

end = time.time()
print(end-start)

{'vendor_name': 'VTS', 'Trip_Pickup_DateTime': '2009-06-14 23:23:00', 'Trip_Dropoff_DateTime': '2009-06-14 23:48:00', 'Passenger_Count': 1, 'Trip_Distance': 17.52, 'Start_Lon': -73.787442, 'Start_Lat': 40.641525, 'Rate_Code': None, 'store_and_forward': None, 'End_Lon': -73.980072, 'End_Lat': 40.742963, 'Payment_Type': 'Credit', 'Fare_Amt': 45.0, 'surcharge': 0.0, 'mta_tax': None, 'Tip_Amt': 9.0, 'Tolls_Amt': 4.15, 'Total_Amt': 58.15}
{'vendor_name': 'VTS', 'Trip_Pickup_DateTime': '2009-06-18 17:35:00', 'Trip_Dropoff_DateTime': '2009-06-18 17:43:00', 'Passenger_Count': 1, 'Trip_Distance': 1.56, 'Start_Lon': -74.009767, 'Start_Lat': 40.722065, 'Rate_Code': None, 'store_and_forward': None, 'End_Lon': -74.005698, 'End_Lat': 40.740187, 'Payment_Type': 'Credit', 'Fare_Amt': 6.5, 'surcharge': 1.0, 'mta_tax': None, 'Tip_Amt': 1.0, 'Tolls_Amt': 0.0, 'Total_Amt': 8.5}
{'vendor_name': 'VTS', 'Trip_Pickup_DateTime': '2009-06-10 18:08:00', 'Trip_Dropoff_DateTime': '2009-06-10 18:27:00', 'Passenger_

In [24]:
import dlt
generators_pipeline=dlt.pipeline(destination='duckdb',dataset_name='generators')

info = generators_pipeline.run(paginated_getter(),
                               table_name="http_download",
                               write_disposition="replace")

print(info)

info = generators_pipeline.run(stream_download_jsonl(url),
                               table_name="stream_download",
                               write_disposition="replace")

print(info)

got page number 1 with 1000 records
got page number 2 with 1000 records
got page number 3 with 1000 records
got page number 4 with 1000 records
got page number 5 with 1000 records
got page number 6 with 1000 records
got page number 7 with 1000 records
got page number 8 with 1000 records
got page number 9 with 1000 records
got page number 10 with 1000 records
got page number 11 with 0 records
Pipeline dlt_colab_kernel_launcher load step completed in 3.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708053867.3630452 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 2.67 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708053899.758

In [26]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path='{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

rides = conn.sql("SELECT * FROM http_download").df()
display(rides)

print("\n\n\n http_download table below:")

passengers = conn.sql("SELECT * FROM http_download").df()
display(passengers)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
│ stream_download     │
└─────────────────────┘




 http_download table below:


,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1708053867.3630452,dTbE2TWcMLZLGw,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1708053867.3630452,caUkwkEv5dsVZw,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1708053867.3630452,xhX/KfwvcqmhFA,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1708053867.3630452,vPCpysiot0S/PQ,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1708053867.3630452,oG9oymSqbkrIIg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40.783522,-73.970690,5.7,1,CASH,40.778560,-73.953660,0.0,0.00,5.70,1.16,2009-06-19 11:28:00+00:00,2009-06-19 11:22:00+00:00,0.0,VTS,1708053867.3630452,2JUxWa92qCx1+Q,NaN
9996,40.777200,-73.964197,4.1,1,CASH,40.779800,-73.974297,0.0,0.00,4.10,0.89,2009-06-17 07:43:00+00:00,2009-06-17 07:41:00+00:00,0.0,VTS,1708053867.3630452,aq5GRlGX+I8Jag,NaN
9997,40.780172,-73.957617,6.1,1,CASH,40.788388,-73.976758,0.0,0.00,6.10,1.30,2009-06-19 11:46:00+00:00,2009-06-19 11:39:00+00:00,0.0,VTS,1708053867.3630452,ZrY/m/RR04Ch6w,NaN
9998,40.777342,-73.957242,5.7,1,CASH,40.773828,-73.956690,0.0,0.00,6.20,0.97,2009-06-17 04:19:00+00:00,2009-06-17 04:13:00+00:00,0.5,VTS,1708053867.3630452,3HTpg7Lhhw8onw,NaN





 http_download table below:


,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1708053867.3630452,dTbE2TWcMLZLGw,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1708053867.3630452,caUkwkEv5dsVZw,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1708053867.3630452,xhX/KfwvcqmhFA,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1708053867.3630452,vPCpysiot0S/PQ,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1708053867.3630452,oG9oymSqbkrIIg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40.783522,-73.970690,5.7,1,CASH,40.778560,-73.953660,0.0,0.00,5.70,1.16,2009-06-19 11:28:00+00:00,2009-06-19 11:22:00+00:00,0.0,VTS,1708053867.3630452,2JUxWa92qCx1+Q,NaN
9996,40.777200,-73.964197,4.1,1,CASH,40.779800,-73.974297,0.0,0.00,4.10,0.89,2009-06-17 07:43:00+00:00,2009-06-17 07:41:00+00:00,0.0,VTS,1708053867.3630452,aq5GRlGX+I8Jag,NaN
9997,40.780172,-73.957617,6.1,1,CASH,40.788388,-73.976758,0.0,0.00,6.10,1.30,2009-06-19 11:46:00+00:00,2009-06-19 11:39:00+00:00,0.0,VTS,1708053867.3630452,ZrY/m/RR04Ch6w,NaN
9998,40.777342,-73.957242,5.7,1,CASH,40.773828,-73.956690,0.0,0.00,6.20,0.97,2009-06-17 04:19:00+00:00,2009-06-17 04:13:00+00:00,0.5,VTS,1708053867.3630452,3HTpg7Lhhw8onw,NaN


In [29]:
import dlt
import duckdb

data = [
    {
        "vendor_name": "VTS",
		"record_hash": "b00361a396177a9cb410ff61f20015ad",
        "time": {
            "pickup": "2009-06-14 23:23:00",
            "dropoff": "2009-06-14 23:48:00"
        },
        "Trip_Distance": 17.52,
        "coordinates": {
            "start": {
                "lon": -73.787442,
                "lat": 40.641525
            },
            "end": {
                "lon": -73.980072,
                "lat": 40.742963
            }
        },
        "Rate_Code": None,
        "store_and_forward": None,
        "Payment": {
            "type": "Credit",
            "amt": 20.5,
            "surcharge": 0,
            "mta_tax": None,
            "tip": 9,
            "tolls": 4.15,
			"status": "cancelled"
        },
        "Passenger_Count": 2,
        "passengers": [
            {"name": "John", "rating": 4.4},
            {"name": "Jack", "rating": 3.6}
        ],
        "Stops": [
            {"lon": -73.6, "lat": 40.6},
            {"lon": -73.5, "lat": 40.5}
        ]
    },
]

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='taxi_rides')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(data,
                    table_name="rides",
                    write_disposition="merge",
                    primary_key="record_hash")

# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.68 seconds
1 load package(s) were loaded to destination duckdb and into dataset taxi_rides
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708054440.1063635 is LOADED and contains no failed jobs


In [37]:

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path='{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Rides table below: Note the times are property typed")
rides = conn.sql("SELECT * FROM rides").df()
display(rides)

print("\n\n\n Passenger table:")
passengers = conn.sql("SELECT * FROM rides__passengers").df()
display(passengers)
print("\n\n\n Stops table:")
stops = conn.sql("SELECT * FROM rides__stops").df()
display(stops)


print("\n\n\n Joined table")

joinned = conn.sql("""
  SELECT *
  FROM rides as r
  left join rides__passengers as rp
    on r._dlt_id = rp._dlt_parent_id
  left join rides__stops as rs
    on r._dlt_id = rs._dlt_parent_id
""").df()

display(joinned)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ rides               │
│ rides__passengers   │
│ rides__stops        │
└─────────────────────┘




 Rides table below: Note the times are property typed


,record_hash,vendor_name,time__pickup,time__dropoff,trip_distance,coordinates__start__lon,coordinates__start__lat,coordinates__end__lon,coordinates__end__lat,payment__type,payment__amt,payment__surcharge,payment__tip,payment__tolls,payment__status,passenger_count,_dlt_load_id,_dlt_id
0,b00361a396177a9cb410ff61f20015ad,VTS,2009-06-14 23:23:00+00:00,2009-06-14 23:48:00+00:00,17.52,-73.787442,40.641525,-73.980072,40.742963,Credit,20.5,0,9,4.15,cancelled,2,1708054440.1063635,7tV+E8r8Hh5cRA





 Passenger table:


,name,rating,_dlt_root_id,_dlt_parent_id,_dlt_list_idx,_dlt_id
0,John,4.4,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,0,5dFQSsOHHSrgeg
1,Jack,3.6,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,1,2ZvxmBffluHdVg





 Stops table:


,lon,lat,_dlt_root_id,_dlt_parent_id,_dlt_list_idx,_dlt_id
0,-73.6,40.6,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,0,QkOwRezb9ZphQA
1,-73.5,40.5,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,1,OLuA3qPLoouwAQ





 Joined table


,record_hash,vendor_name,time__pickup,time__dropoff,trip_distance,coordinates__start__lon,coordinates__start__lat,coordinates__end__lon,coordinates__end__lat,payment__type,...,_dlt_root_id,_dlt_parent_id,_dlt_list_idx,_dlt_id_2,lon,lat,_dlt_root_id_2,_dlt_parent_id_2,_dlt_list_idx_2,_dlt_id_3
0,b00361a396177a9cb410ff61f20015ad,VTS,2009-06-14 23:23:00+00:00,2009-06-14 23:48:00+00:00,17.52,-73.787442,40.641525,-73.980072,40.742963,Credit,...,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,1,2ZvxmBffluHdVg,-73.5,40.5,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,1,OLuA3qPLoouwAQ
1,b00361a396177a9cb410ff61f20015ad,VTS,2009-06-14 23:23:00+00:00,2009-06-14 23:48:00+00:00,17.52,-73.787442,40.641525,-73.980072,40.742963,Credit,...,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,0,5dFQSsOHHSrgeg,-73.5,40.5,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,1,OLuA3qPLoouwAQ
2,b00361a396177a9cb410ff61f20015ad,VTS,2009-06-14 23:23:00+00:00,2009-06-14 23:48:00+00:00,17.52,-73.787442,40.641525,-73.980072,40.742963,Credit,...,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,1,2ZvxmBffluHdVg,-73.6,40.6,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,0,QkOwRezb9ZphQA
3,b00361a396177a9cb410ff61f20015ad,VTS,2009-06-14 23:23:00+00:00,2009-06-14 23:48:00+00:00,17.52,-73.787442,40.641525,-73.980072,40.742963,Credit,...,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,0,5dFQSsOHHSrgeg,-73.6,40.6,7tV+E8r8Hh5cRA,7tV+E8r8Hh5cRA,0,QkOwRezb9ZphQA


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX